# Domain 1 (WASH Guardian) – Conversation + Extraction + Scoring Test Notebook

This notebook helps you **test whether `domain1_agent.py` + `domain1.py` run end-to-end**.

What it does:
- Imports your existing project code (no code changes required)
- Runs a **simulated survey** with scripted respondent answers (so you can test follow-up + NA)
- Prints:
  - Full transcript
  - Extracted flat JSON
  - `Domain1Data` model dump
  - Risk summary

> If your file paths differ, edit the path in the **Setup** cell.


## 1) Setup

Edit `PROJECT_ROOT` if needed so Python can import your modules.


In [1]:
import os, sys, json, asyncio
from pathlib import Path

# TODO: change this if needed
PROJECT_ROOT = Path('..').resolve()  # assumes notebook is in a /notebooks folder inside repo

# Add project root to sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print('PROJECT_ROOT =', PROJECT_ROOT)
print('sys.path[0] =', sys.path[0])


PROJECT_ROOT = /Users/ruigangjiang/Capstone
sys.path[0] = /Users/ruigangjiang/Capstone


## 2) Import your Domain 1 agents/models

This assumes:
- `domain1_agent.py` is importable (adjust import if your module name/location differs)
- `models/domain1.py` defines `Domain1Data`


In [2]:
# Adjust these imports to match your repo structure

from models.domain1 import Domain1Data

# If your file is named differently (e.g., agents/domain1_agent.py), change this import.
import agents.domain1_agent as d1a

print('Imported Domain1Data and domain1_agent OK')


Imported Domain1Data and domain1_agent OK


## 3) A simulated runner (no manual typing)

Your original `run_domain1_survey()` uses `input()` which is awkward in notebooks.

This runner reproduces the same logic but feeds a **scripted list of respondent answers**.


In [3]:
from typing import List, Dict, Any, Optional
import json

async def run_domain1_survey_simulated(responses: List[str]) -> Dict[str, Any]:
    """
    Run Domain 1 survey with scripted respondent answers (no interactive input).

    Returns:
      - transcript_lines
      - extracted_answers
      - domain1
      - risk_summary
    """
    validation_agent = d1a.get_validation_agent()
    extraction_agent = d1a.get_extraction_agent()

    deps = d1a.Domain1SurveyDeps()

    transcript_lines = []
    resp_i = 0
    n_children: Optional[int] = None
    followup_used = [False] * 6

    def should_skip(idx: int) -> bool:
        if idx == 0:
            return False
        if n_children is None:
            return False
        if n_children == 0 and idx in (1,2,3,4):
            return True
        if n_children == 1 and idx in (3,4):
            return True
        return False

    def record(line: str):
        deps.conversation_history.append(line)
        transcript_lines.append(line)

    # Ask Q1 with greeting (same as agent runner)
    q0 = f'Agent: Hello, thank you for participating in our survey today. "{d1a.QUESTIONS[0]}"'
    record(q0)

    q_idx = 0
    while q_idx < 6:
        if should_skip(q_idx):
            record(f"System: Question recorded as NA. Q{q_idx+1}: {d1a.QUESTIONS[q_idx]} | Reason: Not applicable given num_children_under_5={n_children}")
            followup_used[q_idx] = False
            q_idx += 1
            if q_idx < 6 and not should_skip(q_idx):
                record(f'Agent: "{d1a.QUESTIONS[q_idx]}"')
            continue

        # get scripted response
        if resp_i >= len(responses):
            record("System: Scripted responses ended early.")
            break

        user_input = responses[resp_i]
        resp_i += 1
        record(f"Respondent: {user_input}")

        # validate
        current_q_text = f"\"{d1a.QUESTIONS[q_idx]}\""
        vd = await validation_agent.run(
            f"""question_asked: {current_q_text}
respondent_answer: {user_input}
followup_used: {str(followup_used[q_idx]).lower()}"""
        )
        decision = vd.output

        if decision.status == "NEED_FOLLOWUP":
            followup_used[q_idx] = True
            followup_text = (decision.followup or "Could you please clarify?").strip()
            record(f"Agent: {followup_text}")
            continue

        if decision.status == "GIVE_UP":
            record(f"System: Question recorded as NA. Q{q_idx+1}: {d1a.QUESTIONS[q_idx]} | Reason: Unclear after 1 follow-up")
            followup_used[q_idx] = False
            q_idx += 1
            while q_idx < 6 and should_skip(q_idx):
                record(f"System: Question recorded as NA. Q{q_idx+1}: {d1a.QUESTIONS[q_idx]} | Reason: Not applicable given num_children_under_5={n_children}")
                q_idx += 1
            if q_idx < 6:
                record(f'Agent: "{d1a.QUESTIONS[q_idx]}"')
            continue

        # OK
        followup_used[q_idx] = False

        if q_idx == 0:
            n_children = d1a._extract_int_0_2(user_input)
            # after Q1 answer, ask next question
            q_idx += 1
            while q_idx < 6 and should_skip(q_idx):
                record(f"System: Question recorded as NA. Q{q_idx+1}: {d1a.QUESTIONS[q_idx]} | Reason: Not applicable given num_children_under_5={n_children}")
                q_idx += 1
            if q_idx < 6:
                record(f'Agent: "{d1a.QUESTIONS[q_idx]}"')
            continue

        q_idx += 1
        while q_idx < 6 and should_skip(q_idx):
            record(f"System: Question recorded as NA. Q{q_idx+1}: {d1a.QUESTIONS[q_idx]} | Reason: Not applicable given num_children_under_5={n_children}")
            q_idx += 1
        if q_idx < 6:
            record(f'Agent: "{d1a.QUESTIONS[q_idx]}"')

    # Finish
    record("Agent: SURVEY_COMPLETE")

    # Extract
    conversation_text = "\n".join(deps.conversation_history)
    extraction_result = await extraction_agent.run(
        f"Extract the household data from this conversation:\n\n{conversation_text}"
    )
    extracted_answers = extraction_result.output or {}
    domain1 = Domain1Data.from_answers(extracted_answers, strict_len=False)
    risk_summary = domain1.get_risk_summary()

    return {
        "transcript_lines": transcript_lines,
        "extracted_answers": extracted_answers,
        "domain1": domain1,
        "risk_summary": risk_summary,
    }

## 4) Test case A: Normal answers (no follow-up expected)


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY is missing"
print("✅ OPENAI_API_KEY loaded")

✅ OPENAI_API_KEY loaded


In [5]:
responses_A = [
    "2",                  # Q1
    "18",                 # Q2 (months)
    "No",                 # Q3
    "6",                  # Q4
    "Yes",                # Q5
    "Yes elderly; no immunocompromised; mother mainly takes care.",  # Q6
]

outA = await run_domain1_survey_simulated(responses_A)

print("\n".join(outA["transcript_lines"]))
print("\n--- Extracted flat JSON ---")
print(json.dumps(outA["extracted_answers"], indent=2))
print("\n--- Domain1Data ---")
print(outA["domain1"].model_dump_json(indent=2))
print("\n--- Risk summary ---")
print(json.dumps(outA["risk_summary"], indent=2))


Agent: Hello, thank you for participating in our survey today. "How many children under five years old live in your household?"
Respondent: 2
Agent: "Please tell me the age in months of the first child under five."
Respondent: 18
Agent: "Has the first child shown signs of malnutrition, like weight loss or not growing well?"
Respondent: No
Agent: "Please tell me the age in months of the second child under five. If there is no second child, say 'No second child'."
Respondent: 6
Agent: "Has the second child shown signs of malnutrition, like weight loss or not growing well? If there is no second child, say 'No second child'."
Respondent: Yes
Agent: "Are there any elderly or immunocompromised members in your household, and who mainly takes care of the small children during the day?"
Respondent: Yes elderly; no immunocompromised; mother mainly takes care.
Agent: SURVEY_COMPLETE

--- Extracted flat JSON ---
{
  "num_children_under_5": 2,
  "child1_age": 18,
  "child1_malnutrition": false,
  "

## 5) Test case B: Trigger follow-up once; still invalid → NA

This tests your "one follow-up then give up" behavior.

Example: Q2 asks for months, respondent answers vaguely twice.


In [6]:
responses_B = [
    "1",                     # Q1
    "He's a baby",           # Q2 -> follow-up expected
    "Still small",           # follow-up response -> likely GIVE_UP => NA
    "Yes",                   # Q3
    "No second child",       # Q4 (since only 1 child, agent may still ask; your system allows)
    "No second child",       # Q5
    "No; no; both parents",  # Q6
]

outB = await run_domain1_survey_simulated(responses_B)

print("\n".join(outB["transcript_lines"]))
print("\n--- Extracted flat JSON ---")
print(json.dumps(outB["extracted_answers"], indent=2))
print("\n--- Domain1Data ---")
print(outB["domain1"].model_dump_json(indent=2))
print("\n--- Risk summary ---")
print(json.dumps(outB["risk_summary"], indent=2))


Agent: Hello, thank you for participating in our survey today. "How many children under five years old live in your household?"
Respondent: 1
Agent: "Please tell me the age in months of the first child under five."
Respondent: He's a baby
Agent: Please provide the age in months as a number from 0 to 60.
Respondent: Still small
System: Question recorded as NA. Q2: Please tell me the age in months of the first child under five. | Reason: Unclear after 1 follow-up
Agent: "Has the first child shown signs of malnutrition, like weight loss or not growing well?"
Respondent: Yes
System: Question recorded as NA. Q4: Please tell me the age in months of the second child under five. If there is no second child, say 'No second child'. | Reason: Not applicable given num_children_under_5=1
System: Question recorded as NA. Q5: Has the second child shown signs of malnutrition, like weight loss or not growing well? If there is no second child, say 'No second child'. | Reason: Not applicable given num_ch

## Notes / Troubleshooting

- If the import `import domain1_agent as d1a` fails, your file is in a different folder. Change the import accordingly.
- If you see an error about missing OpenAI credentials, make sure you have your environment variables loaded (e.g., `.env`).
- If the agent loops or asks unexpected questions, paste the transcript here and we can tighten prompts.
